# Dependencias

In [1]:
import pandas as pd
from babel.numbers import format_currency

# Lectura de datos

In [2]:
db = pd.read_csv('synergy_logistics_database.csv', index_col = "register_id")
db.head()

,direction,origin,destination,year,date,product,transport_mode,company_name,total_value
register_id,,,,,,,,,
1,Exports,Japan,China,2015,31/01/15,Cars,Sea,Honda,33000000
2,Exports,Japan,China,2015,01/02/15,Cars,Sea,Honda,16000000
3,Exports,Japan,China,2015,02/02/15,Cars,Sea,Honda,29000000
4,Exports,Japan,China,2015,03/02/15,Cars,Sea,Honda,14000000
5,Exports,Japan,China,2015,04/02/15,Cars,Sea,Honda,17000000


# Análisis del dataframe

In [3]:
db.shape # contamos el número de registros y campos

(19056, 9)

In [4]:
db.isnull().values.any() # Buscamos por algún valor nulo, ya que eso generaría sesgo

False

In [5]:
print(db.dtypes)

direction         object
origin            object
destination       object
year               int64
date              object
product           object
transport_mode    object
company_name      object
total_value        int64
dtype: object


# Preguntas de Interés

## Rutas de Importación y Exportación

In [6]:
db['Route'] = db['origin'].str.cat(db['destination'], sep = '-')
db.head()

,direction,origin,destination,year,date,product,transport_mode,company_name,total_value,Route
register_id,,,,,,,,,,
1,Exports,Japan,China,2015,31/01/15,Cars,Sea,Honda,33000000,Japan-China
2,Exports,Japan,China,2015,01/02/15,Cars,Sea,Honda,16000000,Japan-China
3,Exports,Japan,China,2015,02/02/15,Cars,Sea,Honda,29000000,Japan-China
4,Exports,Japan,China,2015,03/02/15,Cars,Sea,Honda,14000000,Japan-China
5,Exports,Japan,China,2015,04/02/15,Cars,Sea,Honda,17000000,Japan-China


In [7]:
print(db['Route'].value_counts())

South Korea-Vietnam    497
Netherlands-Belgium    437
USA-Netherlands        436
Japan-Mexico           385
China-Mexico           351
                      ... 
France-Russia           13
Japan-India              8
France-Austria           7
Brazil-Germany           6
Malaysia-Singapore       4
Name: Route, Length: 172, dtype: int64


In [8]:
dbimports = db.loc[db['direction'] == 'Imports']
dbexports = db.loc[db['direction'] == 'Exports']

In [9]:
dbexports['Route'].groupby(db['direction']).value_counts().head(10)

direction  Route              
Exports    South Korea-Vietnam    497
           Netherlands-Belgium    437
           USA-Netherlands        436
           China-Mexico           330
           Japan-Brazil           306
           Germany-France         299
           South Korea-Japan      279
           Australia-Singapore    273
           Canada-Mexico          261
           China-Spain            250
Name: Route, dtype: int64

In [10]:
dbimports['Route'].groupby(db['direction']).value_counts().head(10)

direction  Route                     
Imports    Singapore-Thailand            273
           Germany-China                 233
           China-Japan                   210
           Japan-Mexico                  206
           China-Thailand                200
           Malaysia-Thailand             195
           Spain-Germany                 142
           Mexico-USA                    122
           China-United Arab Emirates    114
           Brazil-China                  113
Name: Route, dtype: int64

## Ingresos por medio de transporte

In [11]:
"""Forma no tan elegante de encontrar los ingresos por cada medio de transporte"""

dbt = db[["transport_mode","total_value"]]
tm = dbt.groupby("transport_mode").sum()
tm.sort_values(by = ['total_value'], ascending = False).style.format('${0:,.2f}')

,total_value
transport_mode,
Sea,"$100,530,622,000.00"
Rail,"$43,628,043,000.00"
Air,"$38,262,147,000.00"
Road,"$33,270,486,000.00"


In [12]:
"""Este es el valor total de importaciones y exportaciones"""
total = tm['total_value'].sum()
currency = "${:,.2f}".format(total)
eighty_percent = total*0.8
currency2 = "${:,.2f}".format(eighty_percent)
print(f'El total de las exportaciones e importaciones es de:',currency)
print(f'El 80% de los ingresos es de:', currency2)

El total de las exportaciones e importaciones es de: $215,691,298,000.00
El 80% de los ingresos es de: $172,553,038,400.00


### Ingresos por cada medio de transporte (Exportaciones)

In [13]:
dbte = dbexports[["transport_mode","total_value"]]
tme = dbte.groupby("transport_mode").sum()
tme.sort_values(by = ['total_value'], ascending = False).style.format('${0:,.2f}')

,total_value
transport_mode,
Sea,"$65,592,622,000.00"
Rail,"$34,505,043,000.00"
Air,"$32,785,147,000.00"
Road,"$27,280,486,000.00"


### Ingresos por cada medio de transporte (Importaciones)

In [14]:
dbti = dbimports[["transport_mode","total_value"]]
tmi = dbti.groupby("transport_mode").sum()
tmi.sort_values(by = ['total_value'], ascending = False).style.format('${0:,.2f}')

,total_value
transport_mode,
Sea,"$34,938,000,000.00"
Rail,"$9,123,000,000.00"
Road,"$5,990,000,000.00"
Air,"$5,477,000,000.00"


## Valor total de Importaciones y exportaciones

### Valor total de las exportaciones

In [15]:
dbt1 = dbexports[["Route","total_value"]]
tm1 = dbt1.groupby("Route").sum()
tm1.sort_values(by = ['total_value'], ascending = False).style.format('${0:,.2f}')

,total_value
Route,
China-Mexico,"$12,250,000,000.00"
Canada-Mexico,"$8,450,000,000.00"
South Korea-Vietnam,"$6,877,007,000.00"
France-Belgium,"$5,538,069,000.00"
France-United Kingdom,"$5,427,000,000.00"
China-South Korea,"$4,790,000,000.00"
USA-Mexico,"$4,710,000,000.00"
South Korea-Japan,"$4,594,000,000.00"
Germany-Italy,"$4,541,000,000.00"


In [16]:
total1 = tm1['total_value'].sum()
currency3 = "${:,.2f}".format(total1)
eighty_percent1 = total1*0.8
currency4 = "${:,.2f}".format(eighty_percent1)
p_general = total1*100/total
moneda = "${:,.2f}".format(p_general)
print(f'El total de las exportaciones es de:',currency3)
#print(f'El 80% de los ingresos es de:', currency4)
print(f'Y representa el {round(p_general,2)} % de los ingresos generales')

El total de las exportaciones es de: $160,163,298,000.00
Y representa el 74.26 % de los ingresos generales


### Valor total de las importaciones

In [17]:
dbt2 = dbimports[["Route","total_value"]]
tm2 = dbt2.groupby("Route").sum()
tm2.sort_values(by = ['total_value'], ascending = False).style.format('${0:,.2f}')

,total_value
Route,
Singapore-Thailand,"$4,017,000,000.00"
Japan-Mexico,"$3,918,000,000.00"
China-Thailand,"$3,831,000,000.00"
Malaysia-Thailand,"$3,482,000,000.00"
China-United Arab Emirates,"$3,357,000,000.00"
China-Japan,"$3,237,000,000.00"
Mexico-USA,"$2,365,000,000.00"
Japan-United Arab Emirates,"$2,238,000,000.00"
Spain-Germany,"$2,044,000,000.00"


In [18]:
total2 = tm2['total_value'].sum()
currency5 = "${:,.2f}".format(total2)
eighty_percent2 = total2*0.8
currency6 = "${:,.2f}".format(eighty_percent2)
p_generali = total2*100/total
monedita = "${:,.2f}".format(p_generali)
print(f'El total de las importaciones es de:',currency5)
#print(f'El 80% de los ingresos es de:', currency6)
print(f'Y representa el {round(p_generali,2)} % de los ingresos generales')

El total de las importaciones es de: $55,528,000,000.00
Y representa el 25.74 % de los ingresos generales


## Ingresos por país

### Exportaciones

In [19]:
dbta = dbexports[["origin","total_value"]]
tma = dbta.groupby("origin").sum()
tma.rename( columns={ 'total_value' : 'total_exports' } ,inplace=True)
tma.sort_values(by = ['total_exports'], ascending = False).style.format('${0:,.2f}')

,total_exports
origin,
China,"$32,977,046,000.00"
France,"$18,614,332,000.00"
USA,"$18,355,306,000.00"
South Korea,"$14,621,146,000.00"
Russia,"$13,223,000,000.00"
Japan,"$11,948,976,000.00"
Germany,"$11,343,233,000.00"
Canada,"$10,684,000,000.00"
Italy,"$5,366,684,000.00"


In [20]:
tma.shape

(20, 1)

### Importaciones

In [21]:
dbtgh = dbimports[["origin","total_value"]]
tmgh = dbtgh.groupby("origin").sum()
tmgh.rename( columns={ 'total_value' : 'total_imports' } ,inplace=True)
tmgh.sort_values(by = ['total_imports'], ascending = False).style.format('${0:,.2f}')

,total_imports
origin,
China,"$12,233,000,000.00"
Japan,"$8,094,000,000.00"
USA,"$5,291,000,000.00"
Mexico,"$4,621,000,000.00"
Germany,"$4,250,000,000.00"
Singapore,"$4,017,000,000.00"
South Korea,"$3,889,000,000.00"
Malaysia,"$3,560,000,000.00"
Spain,"$3,001,000,000.00"


### Cruce de información

In [22]:
newdf = pd.merge(tma,tmgh, on = 'origin')

In [23]:
sum_column = newdf['total_exports']+newdf['total_imports']
newdf['Total'] = sum_column
newdf.sort_values(by = ['Total'], ascending = False).style.format('${0:,.2f}')

,total_exports,total_imports,Total
origin,,,
China,"$32,977,046,000.00","$12,233,000,000.00","$45,210,046,000.00"
USA,"$18,355,306,000.00","$5,291,000,000.00","$23,646,306,000.00"
Japan,"$11,948,976,000.00","$8,094,000,000.00","$20,042,976,000.00"
France,"$18,614,332,000.00","$1,316,000,000.00","$19,930,332,000.00"
South Korea,"$14,621,146,000.00","$3,889,000,000.00","$18,510,146,000.00"
Germany,"$11,343,233,000.00","$4,250,000,000.00","$15,593,233,000.00"
Russia,"$13,223,000,000.00","$851,000,000.00","$14,074,000,000.00"
Canada,"$10,684,000,000.00","$569,000,000.00","$11,253,000,000.00"
Italy,"$5,366,684,000.00","$1,268,000,000.00","$6,634,684,000.00"


In [24]:
intento = dict(zip(newdf.index,newdf.Total))
sorted_by_value = sorted(intento.items(), key = lambda v: v[1], reverse = True)
sorted_by_value

[('China', 45210046000),
 ('USA', 23646306000),
 ('Japan', 20042976000),
 ('France', 19930332000),
 ('South Korea', 18510146000),
 ('Germany', 15593233000),
 ('Russia', 14074000000),
 ('Canada', 11253000000),
 ('Italy', 6634684000),
 ('Spain', 6419000000),
 ('Mexico', 6040755000),
 ('Singapore', 4017684000),
 ('United Kingdom', 3025612000),
 ('Brazil', 2763000000),
 ('Australia', 2570000000)]

In [25]:
sumita = 0
for k in sorted_by_value:
    if sumita <= eighty_percent:
        moneda1 = "${:,.2f}".format(k[1])
        print(f'{k[0]} tuvo ingresos por: {moneda1} que representa el {round(k[1]/total*100,2)}% de las operaciones')
        sumita += k[1]
moneda2 = "${:,.2f}".format(sumita)
print(f'\nEstos países generaron ingresos por: {moneda2} que representan el {round(sumita/total*100,2)}% del total de los ingresos')

China tuvo ingresos por: $45,210,046,000.00 que representa el 20.96% de las operaciones
USA tuvo ingresos por: $23,646,306,000.00 que representa el 10.96% de las operaciones
Japan tuvo ingresos por: $20,042,976,000.00 que representa el 9.29% de las operaciones
France tuvo ingresos por: $19,930,332,000.00 que representa el 9.24% de las operaciones
South Korea tuvo ingresos por: $18,510,146,000.00 que representa el 8.58% de las operaciones
Germany tuvo ingresos por: $15,593,233,000.00 que representa el 7.23% de las operaciones
Russia tuvo ingresos por: $14,074,000,000.00 que representa el 6.53% de las operaciones
Canada tuvo ingresos por: $11,253,000,000.00 que representa el 5.22% de las operaciones
Italy tuvo ingresos por: $6,634,684,000.00 que representa el 3.08% de las operaciones

Estos países generaron ingresos por: $174,894,723,000.00 que representan el 81.09% del total de los ingresos
